In [ ]:
# ✅ Step 1: Install & Import dependencies
import time
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.optim as optim

# ✅ Step 2: Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ✅ Step 3: Load and transform CIFAR-10 dataset
print("Loading CIFAR-10 dataset...")
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images for AlexNet
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

# ✅ Step 4: Define AlexNet model
print("Building AlexNet model...")

class AlexNet(nn.Module):
    def __init__(self, num_classes=10):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),

            nn.Conv2d(96, 256, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),

            nn.Conv2d(256, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),

            nn.Conv2d(384, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),

            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

model = AlexNet().to(device)

# ✅ Step 5: Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# ✅ Step 6: Train model (short run with progress + timing)
num_epochs = 5  # Keep small for Colab/test speed
print("Starting training...\n")
start_time = time.time()

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch_idx, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if (batch_idx + 1) % 100 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx+1}/{len(train_loader)}], Loss: {loss.item():.4f}")

    print(f"Epoch [{epoch+1}] completed. Average loss: {running_loss / len(train_loader):.4f}")

print(f"\n✅ Training complete in {(time.time() - start_time):.2f} seconds")

# ✅ Step 7: Evaluate model
print("\nEvaluating on test set...")
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"✅ Test Accuracy: {accuracy:.2f}%")


Using device: cuda
Loading CIFAR-10 dataset...
Building AlexNet model...
Starting training...

Epoch [1/5], Step [100/782], Loss: 1.9554
Epoch [1/5], Step [200/782], Loss: 1.9504
Epoch [1/5], Step [300/782], Loss: 1.7165
Epoch [1/5], Step [400/782], Loss: 1.5879
Epoch [1/5], Step [500/782], Loss: 1.6369
Epoch [1/5], Step [600/782], Loss: 1.7471
Epoch [1/5], Step [700/782], Loss: 1.5321
Epoch [1] completed. Average loss: 1.8296
Epoch [2/5], Step [100/782], Loss: 1.7345
Epoch [2/5], Step [200/782], Loss: 1.8862
Epoch [2/5], Step [300/782], Loss: 1.5352
Epoch [2/5], Step [400/782], Loss: 1.4763
Epoch [2/5], Step [500/782], Loss: 1.5768
Epoch [2/5], Step [600/782], Loss: 1.1815
Epoch [2/5], Step [700/782], Loss: 1.4754
Epoch [2] completed. Average loss: 1.5242
Epoch [3/5], Step [100/782], Loss: 1.5689
Epoch [3/5], Step [200/782], Loss: 1.5747
Epoch [3/5], Step [300/782], Loss: 1.3403
Epoch [3/5], Step [400/782], Loss: 1.3503
Epoch [3/5], Step [500/782], Loss: 1.5293
Epoch [3/5], Step [600/